In [1]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from fastapi.responses import JSONResponse
from concurrent.futures import ThreadPoolExecutor
from typing import List
import os
import cv2
import face_recognition

app = FastAPI()

def find_all_the_encodings():
    path = 'faces'
    images = []
    Names = []
    myList = os.listdir(path)

    for cl in myList:
        curImg = cv2.imread(f'{path}/{cl}')
        images.append(curImg)
        Names.append(os.path.splitext(cl)[0])

    def findEncodings(images):
        encodeList = []
        for img in images:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            face_encodings = face_recognition.face_encodings(img)

            if face_encodings:
                encode = face_encodings[0]
                encodeList.append(encode)
            else:
                print("No face found in an image.")
                # If you want to handle the case of no face found, you can append a placeholder value or skip the image.

        return encodeList

    encodeListKnown = findEncodings(images)
    print('Encoding Complete')
    return encodeListKnown, Names

def process_frame(frame_file, output_folder, known_face_encodings, known_face_names):
    frame_path = os.path.join(output_folder, frame_file)
    frame_image = face_recognition.load_image_file(frame_path)
    face_locations = face_recognition.face_locations(frame_image)
    face_encodings = face_recognition.face_encodings(frame_image, face_locations)

    recognized_names = []

    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_dis = face_recognition.face_distance(known_face_encodings, face_encoding)

        match_index = None
        if matches:
            match_index = matches.index(True)

        if match_index is not None:
            name = known_face_names[match_index].upper()
            recognized_names.append(name)

            y, w, h, x = face_location

            # Draw a rectangle around the face
            cv2.rectangle(frame_image, (x, y, x + w, y + h), (0, 255, 0), 2)

            # Display the name on the image
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame_image, name, (x + w // 2 - len(name) * 10, y - 10), font, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    if recognized_names:
        # Save the modified frame as an image with a unique name
        result_filename = os.path.join(output_folder, f"result_{frame_file}")
        cv2.imwrite(result_filename, frame_image)

    return recognized_names


def video_to_frames(video_path, output_folder, target_fps=2):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video information
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video Information - FPS: {fps}, Frame Count: {frame_count}, Resolution: {width}x{height}")

    # Calculate the frame step based on the desired target FPS
    target_frame_step = max(1, int(fps / target_fps))  # Ensure target_frame_step is at least 1

    # Loop through each frame and save it as an image
    for frame_number in range(0, frame_count, target_frame_step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame as an image
        frame_filename = os.path.join(output_folder, f"frame_{frame_number:04d}.jpg")
        cv2.imwrite(frame_filename, frame)

    # Release the video capture object
    cap.release()

if __name__ == "__main__":
    video_path = 'new.mp4'
    output_folder = 'output_frames'
    target_fps = 2

    video_to_frames(video_path, output_folder, target_fps)
    
    known_face_encodings, known_face_names = find_all_the_encodings()

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor() as executor:
        for frame_file in os.listdir(output_folder):
            if frame_file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                executor.submit(process_frame, frame_file, output_folder, known_face_encodings, known_face_names)

                
@app.post("/process_video")
async def process_video(video_file: UploadFile = File(...)):
    # Save the uploaded video file
    video_path = "uploaded_video.mp4"
    with open(video_path, "wb") as video_file_object:
        video_file_object.write(video_file.file.read())

    output_folder = 'output_frames'
    target_fps = 2

    # Convert video to frames
    video_to_frames(video_path, output_folder, target_fps)

    # Load known face encodings
    known_face_encodings, known_face_names = find_all_the_encodings()

    # Process frames with ThreadPoolExecutor for parallel processing
    recognized_names_list = []
    with ThreadPoolExecutor() as executor:
        for frame_file in os.listdir(output_folder):
            if frame_file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                recognized_names = executor.submit(process_frame, frame_file, output_folder, known_face_encodings, known_face_names)
                recognized_names_list.append(recognized_names.result())

    return JSONResponse(content={"recognized_names": recognized_names_list})


Video Information - FPS: 30.0, Frame Count: 1884, Resolution: 720x1280
Encoding Complete


In [1]:
@app.post("/process_video")
async def process_video(video_file: UploadFile = File(...)):
    # Save the uploaded video file
    video_path = "uploaded_video.mp4"
    with open(video_path, "wb") as video_file_object:
        video_file_object.write(video_file.file.read())

    output_folder = 'output_frames'
    target_fps = 2

    # Convert video to frames
    video_to_frames(video_path, output_folder, target_fps)

    # Load known face encodings
    known_face_encodings, known_face_names = find_all_the_encodings()

    # Process frames with ThreadPoolExecutor for parallel processing
    recognized_names_list = []
    with ThreadPoolExecutor() as executor:
        for frame_file in os.listdir(output_folder):
            if frame_file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                recognized_names = executor.submit(process_frame, frame_file, output_folder, known_face_encodings, known_face_names)
                recognized_names_list.append(recognized_names.result())

    return JSONResponse(content={"recognized_names": recognized_names_list})


NameError: name 'app' is not defined